##Thank You Uysim for the lesson

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [ ]:
fast_run=False
image_width=128
image_height=128
image_size=(image_width, image_height)
image_channels = 3

In [ ]:
#Prepare Training DAta
import zipfile
with zipfile.ZipFile('../input/dogs-vs-cats/train.zip', 'r') as zip_ref:
    zip_ref.extractall('../output/kaggle/working/')

In [ ]:
filenames=os.listdir("../output/kaggle/working/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filenames' : filenames,
    'categories': categories
})
df.head()

In [ ]:
df['categories'].value_counts().plot.bar()

In [ ]:
sample = random.choice(filenames)
image = load_img('../output/kaggle/working/train/'+sample)
plt.imshow(image)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(image_width,image_height, image_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
earlystopping = EarlyStopping(patience=10)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2,verbose=1,factor=0.5,min_lr=0.00001)

In [ ]:
callbacks=[earlystopping,learning_rate_reduction]

In [ ]:
df['categories'] = df['categories'].replace({0: 'cat', 1:'dog'})

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
train_df['categories'].value_counts().plot.bar()

In [ ]:
validate_df['categories'].value_counts().plot.bar()

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=15, rescale=1./255, shear_range=0.1, zoom_range=0.2,horizontal_flip=True, width_shift_range=0.1,height_shift_range=0.1)
train_generator = train_datagen.flow_from_dataframe(train_df,"../output/kaggle/working/train/", x_col='filenames', y_col='categories', target_size=image_size,class_mode='categorical',batch_size=batch_size)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator =validation_datagen.flow_from_dataframe(validate_df,"../output/kaggle/working/train/", x_col='filenames', y_col='categories', target_size=image_size,class_mode='categorical',batch_size=batch_size)

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
with tpu_strategy.scope():
    model = Sequential()

    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(image_width,image_height, image_channels)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
epochs=3 if fast_run else 50
history= model.fit_generator(train_generator,epochs=epochs, validation_data=validation_generator,validation_steps=total_validate//batch_size,steps_per_epoch=total_train//batch_size,callbacks=callbacks)

In [ ]:
model.save_weights("model1.h5")

In [ ]:
#virtualize training
fig, (ax1,ax2)= plt.subplots(2,1,figsize=(12,12))
ax1.plot(history.history['loss'], color='b', label='Training Loss')
ax1.plot(history.history['val_loss'], color='r', label = "Validation Loss")
ax1.set_xticks(np.arange(1,epochs,1))

ax2.plot(history.history['accuracy'], color='b', label='Training Accuracy')
ax2.plot(history.history['val_accuracy'], color='r', label = "Validation Accuracy")
ax2.set_xticks(np.arange(1,epochs,1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()